In [46]:
from pymongo import MongoClient
from configparser import ConfigParser
from pprint import pprint

from metatlas import make_xyz_from_stored_data
"""
Creates a connection to the mongodb specified by the config file DB_CONFIG.
The name of the database is 'fireworks' with
collection names 'fireworks', 'workflows', 'launches', and 'fw_id_assigner'
"""
DB_CONFIG = 'qm9_local.ini'
config = ConfigParser()
config.read(DB_CONFIG)
creds = config['db']

client = MongoClient(host=creds['host'], 
                     username=creds['username'],
                     password=creds['password'],
                     authSource=creds['name'])

db = client['fireworks']

print db.collection_names()

[u'fireworks', u'workflows', u'launches', u'fw_id_assigner']


/home/bkrull/bin/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.


## Getting the data out

The fw_ids are unique across the whole database (i.e. 1 molecule <-> 1 fw_id), however, in the 'fireworks' collection, there is no information about stored data. This information is stored in the 'launches' collection. Single fireworks can have multiple launches (which you'll notice for all of them but an excessive number of archived launches for fw_id = [1, 2] because I used these for testing). 

In [58]:
fw = db['fireworks'].find_one({'fw_id': 3})

pprint(fw)

{u'_id': ObjectId('5aaa072b85e51e02812a16dd'),
 u'archived_launches': [3, 133997],
 u'created_on': u'2018-03-15T05:39:55.283749',
 u'fw_id': 3,
 u'launches': [134002],
 u'name': u'/home/bkrull/Documents/data/qm9/dsgdb9nsd_004178',
 u'spec': {u'_tasks': [{u'_fw_name': u'ProtonateMolecule',
                        u'xyzparent': u'/home/bkrull/Documents/data/qm9/dsgdb9nsd_004178.xyz'}]},
 u'state': u'COMPLETED',
 u'updated_on': u'2018-05-09T17:14:04.514475'}


With a launch_id in hand (`max(fw['launches'])` -> yields the largest launch_id aka the most recent launch), you can query the 'launches' collection to get the stored data

In [47]:
launch_id = max(fw['launches'])
launch = db['launches'].find_one({'launch_id': launch_id})
pprint(launch)

## To make an xyz trajectory from stored_data

In [56]:
parent = launch['action']['stored_data']['parent']
children = launch['action']['stored_data']['protonated_children']

In [59]:
atoms = parent['atom_list']
coords = parent['coords'] # [nsteps, natoms, 3]
energies = parent['energies'] # [nsteps]

fname = fw['name'].split('/')[-1]
with open(fname+'.xyz', 'w') as f:
    for step in range(len(coords)):
        f.write(make_xyz_from_stored_data(atoms, coords[step], energies[step]))